In [ ]:
import shutil
import hashlib
import unittest
from pathlib import Path
from datetime import datetime
import joblib
from portfawn.portfolio import BackTest, BackTestAnalysis
from tests.utils import get_normal_param

%matplotlib inline

## Setting the Parameters
The first step to optimize portfolios is to set the configurations such as the list of tickers, backtesting periods, and sampling and optimization paramters.

In [ ]:
# tickers
tickers = {"Global REIT": "REET", 
           "MSCI Emerging Markets": "EEM", 
           "Core S&P Total U.S. Stock Market": "ITOT"}

# date
start_date = datetime.strptime("2015-01-01", "%Y-%m-%d").date()
end_date = datetime.strptime("2020-12-31", "%Y-%m-%d").date()
training_days = 5 * 252
testing_days = 252

# market
risk_free_rate = 0.0

# portfolio
portfolio_fitness = {"MSR": "Max Sharpe-Ratio", 
                     "Equal": "Equal Weight", 'MR': "Maximum Returns"}

optimization_params = {
    "scipy_params": {
        "maxiter": 1000,
        "disp": False,
        "ftol": 1e-10,
    },
    "target_return": 0.1,
    "target_risk": 0.1,
    "weight_bound": (0.0, 1.0),
}
sampling_params = {"type": "standard"}

# system
n_jobs = joblib.cpu_count() - 1

# backtest
backtesting_name = 'test'

# config
backtesting_config = {
    "backtesting_name": backtesting_name,
    "portfolio_fitness": portfolio_fitness,
    "tickers": tickers,
    "start_date": start_date,
    "end_date": end_date,
    "optimization_params": optimization_params,
    "sampling_params": sampling_params,
    "training_days": training_days,
    "testing_days": testing_days,
    "risk_free_rate": risk_free_rate,
    "n_jobs": n_jobs,
}

In [ ]:
portfolio_backtest = BackTest(**backtesting_config)
portfolio_backtest.backtesting_config

In [ ]:
portfolio_backtest.run()

In [ ]:
data_list = []

for profile in portfolio_backtest.profile_backtesting:
    for mode in ['profile_training', 'profile_testing']:
        
        curr = profile[mode]
        d = {'type': curr['type'], 
             'portfolio_fitness': curr['portfolio_fitness'], 
             'date_start': curr['date_start'], 
             'date_end': curr['date_end'], 
             'date': curr['date'], 
             'portfolio_asset_total_return': curr['portfolio_asset_total_return'].to_dict(), 
             'portfolio_asset_mean_std': curr['portfolio_asset_mean_std'].to_dict(), 
             'asset_weights_dict': curr['asset_weights_dict'], 
             'execution_time': curr['execution_time'], 
             'optimization_params': curr['optimization_params'], 
             'sampling_params': curr['sampling_params'], 
             'mode': mode,
             }
        
        data_list.append(d)
    

import pandas as pd
profile_df = pd.DataFrame(data_list)
profile_df

In [ ]:
hash = hashlib.md5(
    "".join([str(i) for i in backtesting_config.values()]).encode("utf-8")
).hexdigest()[0:6]
dirname = f"results_{hash}"
analysis = BackTestAnalysis(portfolio_backtest, result_path=Path(dirname))

In [ ]:
fig, ax = analysis.plot_returns()

In [ ]:
fig, ax = analysis.plot_cum_returns()

In [ ]:
fig, ax = analysis.plot_dist_returns()

In [ ]:
fig, ax = analysis.plot_corr()

In [ ]:
fig, ax = analysis.plot_cov()

In [ ]:
fig, ax = analysis.plot_asset_weights()

In [ ]:
#fig, ax = analysis.plot_mean_std(annualized=True)

In [ ]:
import numpy as np
import pandas as pd
dd = 252

In [ ]:
market_data = 100 * analysis.portfolio_backtesting.market_data.data_returns.loc[analysis.portfolio_backtesting.start_date:analysis.portfolio_backtesting.end_date]
mean_std = pd.DataFrame(columns=["mean", "std"])
mean_std["mean"] = market_data.mean() 
mean_std["std"] = market_data.std() 

mean_std

In [ ]:
# profile_backtesting_test = [
#             i["profile_training"] for i in analysis.profile_backtesting
#         ].copy()
# returns_df = pd.DataFrame(profile_backtesting_test)[
#             ["date", "portfolio_fitness", "daily_return"]
#         ]

# port_data = analysis.portfolio_returns_df
# port_mean_std = pd.DataFrame(columns=["mean", "std"])
# port_mean_std["mean"] = port_data.mean() * dd
# port_mean_std["std"] = port_data.std() * np.sqrt(dd)

port_mean_std = analysis._mean_std
port_mean_std

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(15, 8))

mean_std.plot.scatter(x="std", y="mean", ax=ax, c='green', s=200, alpha=0.8, label='Assets')
port_mean_std.plot.scatter(x="std", y="mean", ax=ax, c='blue', s=200, alpha=0.8, label='Portfolio')

x_min, x_max = mean_std["std"].min(), mean_std["std"].max()
x_diff = x_max - x_min
y_min, y_max = port_mean_std["mean"].min(), port_mean_std["mean"].max()
y_diff = y_max - y_min
        
        
for i, point in mean_std.iterrows():
    ax.text(
        point["std"] - x_diff * 0.05,
        point["mean"] + y_diff * 0.05,
        i,
        fontsize=14,
    )
for i, point in port_mean_std.iterrows():
    ax.text(
        point["std"] - x_diff * 0.05,
        point["mean"] + y_diff * 0.05,
        i,
        fontsize=14,
    )